# Test web application locally¶ 
This notebook pulls some images and tests them against the local web app running inside the Docker container we made previously.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from testing_utilities import to_img, img_url_to_json, plot_predictions
import requests
from dotenv import get_key, find_dotenv
from azureml._model_management._constants import MMS_WORKSPACE_API_VERSION
from azureml._model_management._util import (get_docker_client, pull_docker_image, get_docker_port, 
                                             container_scoring_call, cleanup_container)
from azureml.core import Workspace
from testing_utilities import img_url_to_json
import json
import docker
%matplotlib inline

In [ ]:
import azureml.core
print(azureml.core.VERSION)

In [ ]:
my_workspace = Workspace.from_config()

In [ ]:
image = my_workspace.images['image1']

In [ ]:
image_name = image.image_location

In [ ]:
registry_name = my_workspace.get_details()['containerRegistry'].split('/')[-1]
query_results = !az acr show -n {registry_name} -g {my_workspace.resource_group} --query loginServer -o tsv
acr_server = query_results.s
query_results = !az acr credential show -n {registry_name} -g {my_workspace.resource_group}
acr_credentials = json.loads(query_results.s)

In [ ]:
dc = get_docker_client(acr_credentials['username'], 
                       acr_credentials['passwords'][0]['value'], 
                       acr_server)

In [ ]:
pull_docker_image(dc, image_name, acr_credentials['username'], acr_credentials['passwords'][0]['value'])

Run the Docker conatainer in the background and open port 80. Notice we are using nvidia-docker and not docker command.

In [ ]:
container_labels = {'containerName': 'pytorchgpu'}
container = dc.containers.run(image_name, 
                                         detach=True, 
                                         ports={'5001/tcp': 80},
                                         labels=container_labels,
                                         runtime='nvidia' )

In [ ]:
# TODO Add wait function
container_logs = container.logs().decode('UTF8')

In [ ]:
%time container_logs.find("Users's init has completed successfully")

In [ ]:
print(container.logs().decode('UTF8'))

In [ ]:
client = docker.APIClient()
details = client.inspect_container(container.id)

In [ ]:
service_ip = details['NetworkSettings']['Ports']['5001/tcp'][0]['HostIp']
service_port = details['NetworkSettings']['Ports']['5001/tcp'][0]['HostPort']

Wait a few seconds for the application to spin up and then check that everything works.

In [ ]:
print('Checking service on {} port {}'.format(service_ip, service_port))
!curl 'http://{service_ip}:{service_port}/'

In [ ]:
IMAGEURL = "https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg"

In [ ]:
plt.imshow(to_img(IMAGEURL))

In [ ]:
jsonimg = img_url_to_json(IMAGEURL)
jsonimg[:100]

In [ ]:
headers = {"content-type": "application/json"}
%time r = requests.post('http://0.0.0.0:80/score', data=jsonimg, headers=headers)
print(r)
r.json()

Let's try a few more images.

In [ ]:
images = (
    "https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/3/3a/Roadster_2.5_windmills_trimmed.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Harmony_of_the_Seas_%28ship%2C_2016%29_001.jpg/1920px-Harmony_of_the_Seas_%28ship%2C_2016%29_001.jpg",
    "http://yourshot.nationalgeographic.com/u/ss/fQYSUbVfts-T7pS2VP2wnKyN8wxywmXtY0-FwsgxpiZv_E9ZfPsNV5B0ER8-bOdruvNfMD5EbP4SznWz4PYn/",
    "https://cdn.arstechnica.net/wp-content/uploads/2012/04/bohol_tarsier_wiki-4f88309-intro.jpg",
    "http://i.telegraph.co.uk/multimedia/archive/03233/BIRDS-ROBIN_3233998b.jpg",
)

In [ ]:
url = "http://0.0.0.0:80/score"
results = [
    requests.post(url, data=img_url_to_json(img), headers=headers) for img in images
]

In [ ]:
plot_predictions(images, results)

In [ ]:
image_data = list(map(img_url_to_json, images))  # Retrieve the images and data

In [ ]:
timer_results = list()
for img in image_data:
    res=%timeit -r 1 -o -q requests.post(url, data=img, headers=headers)
    timer_results.append(res.best)

In [ ]:
timer_results

In [ ]:
print("Average time taken: {0:4.2f} ms".format(10 ** 3 * np.mean(timer_results)))

In [ ]:
container.stop()